In [1]:
from astropy.io import fits
import numpy as np
import astropy.coordinates
import astropy.units as u
from functions_pickle import *
import matplotlib.pyplot as plt
import pfsspy

import sunpy
from sunpy.net import Fido,attrs as a

In [ ]:
jsoc_email = ''

In [ ]:
res = Fido.search(a.Time('2011-04-15T00:00:00', '2011-04-16T00:00:00'),  
                  a.jsoc.Series('hmi.mrdailysynframe_720s'),  
                  a.jsoc.Notify(jsoc_email)
                 )  
downloaded_file = Fido.fetch(res)
downloaded_file

Define new function to correct HMI daily synoptic map into something sunpy can read

In [ ]:
def PrepHMIdaily(hmidailyfilepath, numpix_x=1080, numpix_y=540):
    '''
    Function to correct HMI daily synoptic to sunpy map.
    This also resamples the map into a smaller dimension for quicker calculation.
    '''
    from astropy.io import fits
    import numpy as np
    import sunpy.map
    import astropy.time
    import matplotlib.pyplot as plt
    import astropy.units as u
    from sunpy.coordinates import sun


    #Can load only hmi.mrdailysynframe_polfil using astropy.io.fits
    hdus = fits.open(hmidailyfilepath)[0]
    hdus.verify('fix')
    #Metadata fixing
    #Change sin(deg) to degree
    # hdus.header["CUNIT1"] = 'deg'
    hdus.header["CUNIT2"] = 'degree'
    hdus.header["CDELT1"] *= -1
    print(hdus.header["CDELT2"])
    hdus.header["CDELT2"] = 180 / np.pi * hdus.header["CDELT2"]
    # hdus.header["CRVAL1"] = sun.L0(time=hdus.header["DATE-OBS"],aberration_correction=True).value + 120

    hdus.header["CRLN_OBS"] = hdus.header["CRLN_OBS"] +120
    # hdus.header["RSUN_REF"] = 6.955e8
    # original rsun =. 695500000.0
    del hdus.header["CRDER1"]
    del hdus.header["CRDER2"]
    del hdus.header["CSYSER1"]
    del hdus.header["CSYSER2"]

    hmi_syn_map = sunpy.map.Map(np.nan_to_num(hdus.data), hdus.header)
    time = hmi_syn_map.meta['T_OBS'][0:10].replace('.','-')+'T'+hmi_syn_map.meta['T_OBS'][11:-4]
    print(time)
    
    t_hmi_syn = astropy.time.Time(time, scale='tai')
    hdus.header["DATE-OBS"] = t_hmi_syn.utc.value
    print(hdus.header["CRVAL1"])
    print(sun.L0(time=t_hmi_syn.utc.value).value)

    hmi_syn_map = sunpy.map.Map(np.nan_to_num(hdus.data), hdus.header)
    print(hdus.header["DATE-OBS"])
    # Somehow this weird combination works for correcting the HMI daily synoptic map
    hmi_syn_map.meta["CRVAL1"] = hdus.header['CRLN_OBS'] +hdus.header['CRLT_OBS']
    # Plot settings
    hmi_syn_map.plot_settings['norm'] =plt.Normalize(-200,200)

#    #Resample to reduced computational time
    print(hmi_syn_map.dimensions)
    hmi_syn_map = hmi_syn_map.resample([numpix_x, numpix_y] * u.pix)## [longitude, latitude]

    return hmi_syn_map

Using the above function to generate a sunpy HMI daily synoptic map

In [ ]:
hmi_syn = PrepHMIdaily(downloaded_file[0])

Setting seed points for PFSSpy to trace

In [ ]:
min_gauss = 100
masked_pix_y, masked_pix_x = np.where(np.logical_or(hmi_syn.data >min_gauss,hmi_syn.data <-min_gauss))
seeds = hmi_syn.pixel_to_world(masked_pix_x*u.pix, masked_pix_y*u.pix,).make_3d()
seeds_hpc = seeds.transform_to(aia_submap.coordinate_frame)
in_lon = np.logical_and(seeds_hpc.Tx > aia_submap.bottom_left_coord.Tx,
                        seeds_hpc.Tx < aia_submap.top_right_coord.Tx)
in_lat = np.logical_and(seeds_hpc.Ty > aia_submap.bottom_left_coord.Ty,
                        seeds_hpc.Ty < aia_submap.top_right_coord.Ty)
seeds_eis = seeds[np.where(np.logical_and(in_lon, in_lat))]
seeds_eis = seeds_eis[np.where(np.logical_and(seeds_eis.lon.to_value()<360, seeds_eis.lon.to_value()>270))]

Input synoptic map into pfsspy

In [ ]:
pfss_input = pfsspy.Input(hmi_syn, 70, 2.5)
pfss_output = pfsspy.pfss(peri_input)

In [ ]:
tracer = pfsspy.tracing.FortranTracer()
flines = tracer.trace(SkyCoord(seeds_eis), pfss_output)

Calculating the total field strength along each loop

In [ ]:
from tqdm import tqdm
total_flux=[]

for i in tqdm(flines):
    try:
        total_flux.append(np.sum([(np.sum(b**2))**0.5 for b in i.b_along_fline]))
    except:
        pass